# 如何检查 runnables

:::info 先决条件

本指南假定您熟悉以下概念：
- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)
- [链接 runnables](/docs/how_to/sequence/)

:::

一旦您使用 [LangChain 表达式语言](/docs/concepts/lcel) 创建了一个 runnable，您可能经常想检查它来更好地了解正在发生的事情。本笔记涵盖了一些方法。

本指南介绍了一些您可以以编程方式内省链的内部步骤的方法。如果您有兴趣调试链中的问题，请参阅 [本节](/docs/how_to/debugging)。

首先，让我们创建一个示例链。我们将创建一个执行检索的链：

In [ ]:
%pip install -qU langchain langchain-openai faiss-cpu tiktoken

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## 获取图

您可以使用 `get_graph()` 方法来获取可运行对象的图表示：

In [ ]:
chain.get_graph()

## 打印图表

虽然这样不够直观，但你可以使用 `print_ascii()` 方法来以一种更容易理解的方式显示该图表：

In [5]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## 获取提示词

您可能只想查看在链中使用到的提示词，可以使用 `get_prompts()` 方法：

In [6]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])]

## 下一步

您现已学会了如何自省组合的 LCEL 链。

接下来，请查阅本节中关于 runnables 的其他操作指南，或关于[调试链条](/docs/how_to/debugging)的相关操作指南。